# This notebook analyzes the Zoo sample with the same LRG cuts from Petrillo and SDSS.
(i) The low-z (z < 0.4) LRG colour-magnitude selection of
Eisenstein et al. (2001), adapted to including more sources
(fainter and bluer):
r < 20
|c perp | < 0.2
r < 14 + c par /0.3
where
(1)
c par = 0.7(g − r) + 1.2[(r − i) − 0.18)]
c perp = (r − i) − (g − r)/4.0 − 0.18

(ii) A source size in the r -band larger than the average
FWHM of the PSF of the respective tiles, times a empir-
ical factor to maximize the separation between stars and galaxies.
# Edit 06/05/20 I made a grave mistake here and am redoing these calculations.
# Edit 06/09/20 - Redoing with final number of 16 candidates.
# Edit 07/1/20 - Redoing with 13 candidates. (final? haha)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial.polynomial import polyfit
import pandas as pd
#from matplotlib_venn import venn3, venn3_circles
import matplotlib.lines as lines
from astropy import constants as const
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table
# import ephem
from astropy.cosmology import FlatLambdaCDM
from scipy import stats as sci

## g-band magnitude (AB)

In [3]:
# Opening GAMA LambdarCat data from G09, G12, and G15 (DR3)
hdul = fits.open('/home/shawn/Desktop/gravitational_lensing_research/FITS_Files/LambdarSDSSg.fits')  # open a FITS file
hdul.verify('fix')
photo_g_data = hdul[1].data  # assume the first extension is a table
#print(photo_g_data.columns)

# take ID and mag in g
photo_g_id = photo_g_data.LAMID
# LAMID is GAMA ID where applicable.
photo_g_mag = photo_g_data.ABMagDFA

photo_g_list = list(zip(photo_g_id, photo_g_mag))

photo_g = pd.DataFrame(photo_g_list, columns = ['GAMA_ID', 'g'])
photo_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220158 entries, 0 to 220157
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   GAMA_ID  220158 non-null  object 
 1   g        220158 non-null  float64
dtypes: float64(1), object(1)
memory usage: 3.4+ MB


In [4]:
# GAMA_ID is given in bytes, need to decode. #b'8000'.decode("utf-8")
#photo_g_id.decode("utf-8")
#photo_g['GAMA_ID'] = photo_g['GAMA_ID'].str.decode("utf-8")

In [5]:
# Remove IDs which have "New" in the name
photo_g = photo_g[~photo_g.GAMA_ID.str.contains('New')]
photo_g

,GAMA_ID,g
0,100000,20.594820
1,100001,20.743700
2,100002,18.867937
3,100003,20.013288
4,100004,19.506737
...,...,...
219453,99995,21.020639
219454,99996,21.404068
219455,99997,21.347977
219456,99998,20.773315


In [6]:
# Write IDs as integers
photo_g['GAMA_ID'] = photo_g.GAMA_ID.astype(int)

In [7]:
photo_g

,GAMA_ID,g
0,100000,20.594820
1,100001,20.743700
2,100002,18.867937
3,100003,20.013288
4,100004,19.506737
...,...,...
219453,99995,21.020639
219454,99996,21.404068
219455,99997,21.347977
219456,99998,20.773315


## r-band

In [8]:
# Opening GAMA LambdarCat data from G09, G12, and G15 (DR3)
hdul = fits.open('/home/shawn/Desktop/gravitational_lensing_research/FITS_Files/LambdarSDSSr.fits')  # open a FITS file
hdul.verify('fix')
photo_r_data = hdul[1].data  # assume the first extension is a table
#print(photo_g_data.columns)

# take ID and mag in g
photo_r_id = photo_r_data.LAMID
# LAMID is GAMA ID where applicable.
photo_r_mag = photo_r_data.ABMagDFA

photo_r_list = list(zip(photo_r_id, photo_r_mag))

photo_r = pd.DataFrame(photo_r_list, columns = ['GAMA_ID', 'r'])
photo_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220158 entries, 0 to 220157
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   GAMA_ID  220158 non-null  object 
 1   r        220158 non-null  float64
dtypes: float64(1), object(1)
memory usage: 3.4+ MB


In [9]:
# GAMA_ID is given in bytes, need to decode. #b'8000'.decode("utf-8")
#photo_g_id.decode("utf-8")
#photo_r['GAMA_ID'] = photo_r['GAMA_ID'].str.decode("utf-8")

In [10]:
# Remove IDs which have "New" in the name
photo_r = photo_r[~photo_r.GAMA_ID.str.contains('New')]
photo_r

,GAMA_ID,r
0,100000,19.811590
1,100001,19.662579
2,100002,17.846043
3,100003,19.200987
4,100004,19.207859
...,...,...
219453,99995,19.435972
219454,99996,19.633066
219455,99997,19.507254
219456,99998,19.538841


In [11]:
# Write IDs as integers
photo_r['GAMA_ID'] = photo_r.GAMA_ID.astype(int)

In [12]:
photo_r

,GAMA_ID,r
0,100000,19.811590
1,100001,19.662579
2,100002,17.846043
3,100003,19.200987
4,100004,19.207859
...,...,...
219453,99995,19.435972
219454,99996,19.633066
219455,99997,19.507254
219456,99998,19.538841


## i-band

In [13]:
# Opening GAMA LambdarCat data from G09, G12, and G15 (DR3)
hdul = fits.open('/home/shawn/Desktop/gravitational_lensing_research/FITS_Files/LambdarSDSSi.fits')  # open a FITS file
hdul.verify('fix')
photo_i_data = hdul[1].data  # assume the first extension is a table
#print(photo_g_data.columns)

# take ID and mag in g
photo_i_id = photo_i_data.LAMID
# LAMID is GAMA ID where applicable.
photo_i_mag = photo_i_data.ABMagDFA

photo_i_list = list(zip(photo_i_id, photo_i_mag))

photo_i = pd.DataFrame(photo_i_list, columns = ['GAMA_ID', 'i'])
photo_i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220158 entries, 0 to 220157
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   GAMA_ID  220158 non-null  object 
 1   i        220158 non-null  float64
dtypes: float64(1), object(1)
memory usage: 3.4+ MB


In [14]:
# GAMA_ID is given in bytes, need to decode. #b'8000'.decode("utf-8")
#photo_g_id.decode("utf-8")
#photo_i['GAMA_ID'] = photo_i['GAMA_ID'].str.decode("utf-8")

In [15]:
# Remove IDs which have "New" in the name
photo_i = photo_i[~photo_i.GAMA_ID.str.contains('New')]
photo_i

,GAMA_ID,i
0,100000,19.469633
1,100001,19.226625
2,100002,17.358280
3,100003,18.706486
4,100004,18.875196
...,...,...
219453,99995,18.752949
219454,99996,18.998640
219455,99997,18.886461
219456,99998,19.030018


In [16]:
# Write IDs as integers
photo_i['GAMA_ID'] = photo_i.GAMA_ID.astype(int)
photo_i

,GAMA_ID,i
0,100000,19.469633
1,100001,19.226625
2,100002,17.358280
3,100003,18.706486
4,100004,18.875196
...,...,...
219453,99995,18.752949
219454,99996,18.998640
219455,99997,18.886461
219456,99998,19.030018


## Now I create variables to establish criteria for satisfying Petrillo's LRG classification.

## Now to see if Zoo passes the LRG cuts.

In [17]:
zoo = pd.read_csv('/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/zoo_latest.csv')

In [19]:
zoo[zoo.score > 0.35]

,Unnamed: 0,Unnamed: 0.1,GAMA_ID,RA,DEC,score,z,magphys_mstar,lambdar_log_mstar,lambdar_mstar,lambdar_log_mstar_error,m_half_re,sigma_star,D_lens,theta_e_pm,theta_e_sis
1,1,1,84050,175.79849,0.47814,0.365758,0.1190,5.471000e+10,11.056287,1.138379e+11,0.144809,7.273630e+10,202.447322,421.600202,0.837356,0.592336
3,3,3,70282,179.40308,0.12562,0.655352,0.1074,4.256000e+10,10.978367,9.514082e+10,0.123709,5.812343e+10,192.079710,385.434393,0.782862,0.533221
5,5,5,124486,179.72998,-2.51919,0.426190,0.1435,8.541000e+10,11.169208,1.476412e+11,0.114999,1.006705e+11,218.473025,494.906734,0.909231,0.689826
6,6,6,593852,218.07820,-0.03958,0.386318,0.1308,2.251000e+11,11.554501,3.585094e+11,0.103686,3.051535e+11,283.327458,457.418147,1.646598,1.160170
7,7,7,93310,219.92276,0.50731,0.575139,0.1377,3.094000e+10,10.989346,9.757679e+10,0.116852,5.998958e+10,193.507824,477.920325,0.714242,0.541179
9,9,9,600421,135.49058,0.28417,0.420189,0.0565,2.621000e+10,10.758240,5.731123e+10,0.134361,3.084554e+10,165.570990,214.794361,0.763958,0.396198
10,10,10,574423,135.75994,-0.20038,0.464337,0.0536,5.079000e+10,11.045467,1.110369e+11,0.119129,7.050614e+10,200.974957,204.451449,1.183868,0.583751
11,11,11,324764,137.20387,1.72633,0.422757,0.1044,4.647000e+10,11.063084,1.156335e+11,0.106676,7.417324e+10,203.377784,375.923541,0.895486,0.597793
12,12,12,342699,216.90174,2.13250,0.457088,0.0876,2.845000e+10,10.787631,6.132408e+10,0.125984,3.356846e+10,168.886872,321.432394,0.651487,0.412226


In [20]:
# create variables for the criteria

# join all photometry measurements to GAMA_ID
gr = pd.merge(photo_g, photo_r, on = 'GAMA_ID')
gri = pd.merge(gr, photo_i, on = 'GAMA_ID')

# convert to Johnson magnitude system (or # out addition for AB)
# We want AB.
g = gri.g# + 0.013
r = gri.r# + 0.226
i = gri.i# + 0.296

c_par = 0.7*(g - r) + 1.2*((r - i) - 0.18)
c_perp = (r - i) - (g - r)/4.0 - 0.18

gri['c_par'] = c_par
gri['c_perp'] = c_perp

gri

# this was the huge mistake... really stupid.
# add columns c_par and c_perp to zoo
#zoo['r'] = r
#zoo['c_par'] = c_par
#zoo['c_perp'] = c_perp


,GAMA_ID,g,r,i,c_par,c_perp
0,100000,20.594820,19.811590,19.469633,0.742609,-0.033850
1,100001,20.743700,19.662579,19.226625,1.063929,-0.014327
2,100002,18.867937,17.846043,17.358280,1.084641,0.052289
3,100003,20.013288,19.200987,18.706486,0.946012,0.111426
4,100004,19.506737,19.207859,18.875196,0.392410,0.077943
...,...,...,...,...,...,...
219453,99995,21.020639,19.435972,18.752949,1.712895,0.106857
219454,99996,21.404068,19.633066,18.998640,1.785013,0.011676
219455,99997,21.347977,19.507254,18.886461,1.817457,-0.019388
219456,99998,20.773315,19.538841,19.030018,1.258720,0.020205


In [21]:
# merge zoo with gri
zoo = pd.merge(zoo, gri, on = 'GAMA_ID')
zoo

,Unnamed: 0,Unnamed: 0.1,GAMA_ID,RA,DEC,score,z,magphys_mstar,lambdar_log_mstar,lambdar_mstar,...,m_half_re,sigma_star,D_lens,theta_e_pm,theta_e_sis,g,r,i,c_par,c_perp
0,0,0,485873,217.75015,-1.80042,0.349456,0.0539,4.908000e+10,11.043426,1.105161e+11,...,7.009300e+10,200.698297,205.524664,1.177309,0.582145,16.423378,15.589527,15.164383,0.877869,0.036681
1,1,1,84050,175.79849,0.47814,0.365758,0.1190,5.471000e+10,11.056287,1.138379e+11,...,7.273630e+10,202.447322,421.600202,0.837356,0.592336,17.991753,17.103765,16.721144,0.864737,-0.019375
2,2,2,55245,181.07958,-0.31553,0.314054,0.0936,1.274000e+11,11.350592,2.241773e+11,...,1.696806e+11,246.912657,341.136092,1.421796,0.881111,15.879565,15.267034,14.955459,0.586662,-0.021558
3,3,3,70282,179.40308,0.12562,0.655352,0.1074,4.256000e+10,10.978367,9.514082e+10,...,5.812343e+10,192.079710,385.434393,0.782862,0.533221,18.413406,17.419868,16.975845,1.012304,0.015639
4,4,4,185451,180.28326,-1.61006,0.302985,0.1079,8.222000e+10,11.192692,1.558446e+11,...,1.077103e+11,221.962032,387.013180,1.063533,0.712035,17.746004,16.827835,16.369450,0.976781,0.048843
5,5,5,124486,179.72998,-2.51919,0.426190,0.1435,8.541000e+10,11.169208,1.476412e+11,...,1.006705e+11,218.473025,494.906734,0.909231,0.689826,18.709095,17.625223,17.182020,1.074554,-0.007765
6,6,6,593852,218.07820,-0.03958,0.386318,0.1308,2.251000e+11,11.554501,3.585094e+11,...,3.051535e+11,283.327458,457.418147,1.646598,1.160170,17.762629,16.680119,16.225178,1.087686,0.004313
7,7,7,93310,219.92276,0.50731,0.575139,0.1377,3.094000e+10,10.989346,9.757679e+10,...,5.998958e+10,193.507824,477.920325,0.714242,0.541179,18.771408,17.814560,17.393770,0.958741,0.001578
8,8,8,136604,175.87349,-1.74167,0.316451,0.1060,2.866000e+11,11.663817,4.611236e+11,...,4.179886e+11,305.013103,381.004142,2.111555,1.344563,16.732225,15.738844,15.305708,0.999130,0.004791
9,9,9,600421,135.49058,0.28417,0.420189,0.0565,2.621000e+10,10.758240,5.731123e+10,...,3.084554e+10,165.570990,214.794361,0.763958,0.396198,16.958435,16.209999,15.800235,0.799622,0.042655


In [22]:
# Take Petrillo's 1st set of criteria for LRG (z < 0.4)
zoo_LRG = zoo[(zoo.r < 20) & (abs(zoo.c_perp) < 0.2) & (zoo.r < 14 + zoo.c_par/0.3)]
#zoo_LRG_1 = zoo_LRG_1[abs(zoo_LRG_1.r) < 100]

print(len(zoo_LRG))

zoo_not_LRG = zoo[~zoo.GAMA_ID.isin(zoo_LRG.GAMA_ID)]

print(len(zoo_not_LRG))

8
5


## Only six candidates pass the LRG cuts imposed by Petrillo!
# Edit 06/05/20 - 17 pass the LRG cuts. Lets look at the other 19.

In [20]:
zoo_not_LRG.c_perp > 0.2
# All 19 pass the c_perp aspect

1     False
3     False
5     False
7     False
12    False
13    False
14    False
15    False
Name: c_perp, dtype: bool

In [24]:
zoo_not_LRG['r_minus_cpar'] = zoo_not_LRG.r - (14 + zoo_not_LRG.c_par/0.3)
zoo_not_LRG.r_minus_cpar
# all have r > calculation... Meaning they are not red?

/home/shawn/miniconda3/envs/astroconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


1     0.221308
3     0.045521
5     0.043377
7     0.618755
12    0.106565
Name: r_minus_cpar, dtype: float64

In [25]:
print(np.mean(zoo_not_LRG.r_minus_cpar[zoo_not_LRG.r_minus_cpar > 0]),
np.median(zoo_not_LRG.r_minus_cpar[zoo_not_LRG.r_minus_cpar > 0]))

0.20710530598958385 0.10656489054361984


## All 19 candidates that fail LRG cuts with Lambdar photometry fail the c_par aspect, which is intended to set a luminosity threshold as a sliding function of redshift. It is compared to the r-band magnitude to represent an older stellar population. However, (Eisenstein-2001) "strongly advise[s] the reader that the LRG flags cannot be used to select a volume-weighted sample at z < 0.15... However, the linear color-magnitude cut (eq. [4]) is not a good approximation to the locus of an early-type galaxy at lower redshifts. At z < 0.15, Cut I is too permissive, allowing lower luminosity sources to enter the LRG sample. Hence, to extract LRGs from the MAIN sample at z < 0.15, one must make additional post-spectroscopic cuts." They do not give a "clean prescription for this".
## All 19 fail the the sliding luminosity threshold as a function of redshift by an mean of 0.66 and median of 0.82 mag.
# EDIT. New number is 8 pass, 8 fail.
## All 8 fail the the sliding luminosity threshold as a function of redshift by an mean of 0.39 and median of 0.29 mag.

In [23]:
zoo_not_LRG.Z

1     0.1190
3     0.1074
5     0.1435
7     0.1377
12    0.0876
13    0.0985
14    0.1436
15    0.1221
Name: Z, dtype: float64